In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import csv
import numpy as np

print('Hello Welcome to the model trining process lets start ........................')

input_size = 2
num_out = 1
num_epochs = 10000
learning_rate = 0.00001

def read_csv_as_a_dict(file_path):
    with open(file_path, newline='') as csvfile:
        data = csv.reader(csvfile)
        parameters = next(data)
        units = next(data)
        values = [[]]
        for row in data:
            values += [list(row)]
        values = np.array(values[1:], dtype=float)
        data_dictionary = {}
        for i, parameter in enumerate(parameters[1:], start=1):
            data_dictionary[parameter.replace(' ', '_')] = values[:, i]
    return data_dictionary

def create_model(input_size, num_classes):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_size,)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(num_classes)
    ])
    return model

data_path = "/content/Electric State Input and Output Characteristic Test Data .csv"
data = read_csv_as_a_dict(data_path)
Speed = data['Speed']
Torque = data['Torque']
Efficiency = data['Efficiency_of_System'] / 100

X = np.array([Speed, Torque]).T
Y = np.array([Efficiency]).T

model = create_model(input_size, num_out)
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

# Note: The verbose parameter in model.fit() controls the detail of training log messages:
# - 0 = silent, 1 = progress bar, 2 = one line per epoch.

model.fit(X, Y, epochs=num_epochs, batch_size=32, verbose=1, callbacks=[
    tf.keras.callbacks.LearningRateScheduler(lambda epoch: learning_rate * 0.9 ** (epoch // 10000), verbose=1)
])

# Save the final models after training completes
model.save('Reg_Electric.h5')
tf.saved_model.save(model, 'Reg_Electric')



Streaming output truncated to the last 5000 lines.
Epoch 8751: LearningRateScheduler setting learning rate to 1e-05.
Epoch 8751/10000
7/7 [==============================] - 0s 5ms/step - loss: 0.0119 - lr: 1.0000e-05

Epoch 8752: LearningRateScheduler setting learning rate to 1e-05.
Epoch 8752/10000
7/7 [==============================] - 0s 5ms/step - loss: 0.0129 - lr: 1.0000e-05

Epoch 8753: LearningRateScheduler setting learning rate to 1e-05.
Epoch 8753/10000
7/7 [==============================] - 0s 6ms/step - loss: 0.0084 - lr: 1.0000e-05

Epoch 8754: LearningRateScheduler setting learning rate to 1e-05.
Epoch 8754/10000
7/7 [==============================] - 0s 5ms/step - loss: 0.0093 - lr: 1.0000e-05

Epoch 8755: LearningRateScheduler setting learning rate to 1e-05.
Epoch 8755/10000
7/7 [==============================] - 0s 5ms/step - loss: 0.0074 - lr: 1.0000e-05

Epoch 8756: LearningRateScheduler setting learning rate to 1e-05.
Epoch 8756/10000
7/7 [==========================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
import numpy as np

h5_model = tf.keras.models.load_model('/content/Reg_Electric.h5')

pb_model = tf.saved_model.load('/content/Reg_Electric')

def predict_with_h5_model(input_data):
    input_data = np.array(input_data).reshape(1, 2)  # Reshape input for single prediction
    prediction = h5_model.predict(input_data)
    return prediction


def predict_with_pb_model(input_data):
    input_data = tf.convert_to_tensor(input_data, dtype=tf.float32)  # Convert input to tensor
    input_data = tf.expand_dims(input_data, axis=0)  # Add batch dimension
    prediction = pb_model(input_data)  # Call the model
    return prediction.numpy()  # Convert tensor to numpy array

# Example input values
input_values = [12.5, 30]

h5_prediction = predict_with_h5_model(input_values)
print("Prediction using HDF5 model:", h5_prediction)

pb_prediction = predict_with_pb_model(input_values)
print("Prediction using Protocol Buffer model:", pb_prediction)


1/1 [==============================] - 0s 104ms/step
Prediction using HDF5 model: [[2.6431694]]
Prediction using Protocol Buffer model: [[2.6431694]]


In [ ]:
# Convert the model to tensorflow lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Open the model to view the format
open("moisture_sensor_model.tflite", "wb").write(tflite_model)


14468

In [ ]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model('/content/Electric_state_input.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def qunatization_dataset_gen():
    for _ in range(100000):
        yield [
            np.array(
                [np.random.uniform(0, 10000), np.random.uniform(0, 100)] # Enter the range of inputs the model might have during the inference process
            , dtype=np.float32)
        ]
converter.representative_dataset = qunatization_dataset_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

In [ ]:
import tensorflow as tf
import numpy as np


model = tf.keras.models.load_model('/content/Electric_state_input.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
!xxd -i converted_model.tflite > model_data.h